<a href="https://colab.research.google.com/github/lara-vel-dev/SQL-Bootcamp-MS/blob/main/Caba%C3%B1a_del_Misterio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reporte técnico de la Cabaña del Misterio
## Introducción
Esta base de datos modela un escenario ficticio orientado a experiencias paranormales en la cabaña del misterio, estructurado en torno a visitantes, empleados, productos, ventas y reportes de eventos sobrenaturales. Los queries desarrollados permiten explorar aspectos clave del funcionamiento de este sitio, como la relación entre personal y jerarquías, comportamiento de los clientes, control de inventarios, y la interacción entre eventos misteriosos y consumo.

# Requerimientos iniciales para la conexión a la base de datos

In [ ]:
# Dependencias
import sqlite3
import pandas as pd


In [ ]:
# Conexión a la base de datos
conn = sqlite3.connect('/content/mystery_shack.db')
cursor = conn.cursor()



## 3. Consultas SQL y análisis

### 3.1 Listado de empleados con su jefe (si aplica), ordenados por puesto y fecha de ingreso

In [ ]:
query1 = """
SELECT
    e1.nombre AS empleado,
    e1.puesto,
    e1.fecha_ingreso,
    e2.nombre AS jefe
FROM empleado e1
LEFT JOIN empleado e2 ON e1.jefe_id = e2.id_empleado
ORDER BY e1.puesto, e1.fecha_ingreso;
"""
df_empleados = pd.read_sql_query(query1, conn)
df_empleados


,empleado,puesto,fecha_ingreso,jefe
0,Soos Ramírez,Ayudante de mantenimiento,1997-07-13,Stanley Pines
1,Wendy Corduroy,Cajera,2014-08-03,Stanley Pines
2,Stanley Pines,Gerente,1990-06-01,None


El resultado de la consulta muestra un listado de empleados junto con el nombre de su jefe, ordenados por puesto y fecha de ingreso. En este caso, se observa que Stanley Pines ocupa el puesto de gerente y es el empleado con mayor antigüedad, habiendo ingresado en 1990. Los otros dos empleados, Soos Ramírez y Wendy Corduroy, dependen directamente de él, desempeñándose como ayudante de mantenimiento y cajera, respectivamente. La estructura refleja una organización pequeña con una jerarquía clara, donde el gerente supervisa directamente a sus subordinados. El ordenamiento por puesto y antigüedad permite identificar fácilmente la cadena de mando y la experiencia de cada empleado dentro de la empresa, lo cual es útil para la gestión de recursos humanos y la planificación organizacional.

### 3.2 Visitantes que han realizado una compra, con los productos adquiridos y la fecha de la venta

In [ ]:
query2 = """
SELECT
    v.nombre AS visitante,
    p.nombre AS producto,
    ve.fecha_venta
FROM venta ve
JOIN visitante v ON ve.id_visitante = v.id_visitante
JOIN producto p ON ve.id_producto = p.id_producto
ORDER BY ve.fecha_venta;
"""
df_compras = pd.read_sql_query(query2, conn)
df_compras

,visitante,producto,fecha_venta
0,Tyler Ruedabueno,Camiseta de puma,2025-02-05
1,Tyler Ruedabueno,Camiseta de pantera,2025-02-05
2,Dipper Pines,Camiseta misteriosa,2025-03-11
3,Mabel Pines,Peluche de pato,2025-03-11
4,Mabel Pines,Cabezón de Stan Pines,2025-03-11
5,Candy Chiu,Peluche de pato,2025-03-11
6,Grenda,Peluche de pato,2025-03-11
7,Grenda,Llavero de ovni,2025-03-11
8,Grenda,Bola de nieve cabaña del misterio,2025-03-11
9,Robbie Valentino,Bola de nieve cabaña del misterio,2025-03-12


El listado de visitantes que han realizado compras revela una diversidad de productos adquiridos en distintas fechas, lo que sugiere un flujo constante de clientes activos en la cabaña del misterio. Tyler Ruedabueno destaca por haber comprado dos camisetas el mismo día, mientras que los hermanos Dipper y Mabel Pines realizaron compras variadas, que incluyen desde camisetas misteriosas hasta artículos de colección como peluches y figuras. También se observa que otros visitantes, como Candy Chiu, Grenda y Robbie Valentino, han adquirido souvenirs relacionados con la temática del lugar, tales como llaveros y bolas de nieve. Este comportamiento indica un interés por productos temáticos que refuerzan la experiencia del visitante, a la vez que refleja patrones de compra relacionados con eventos o visitas grupales en fechas específicas. Estos datos permiten comprender mejor las preferencias de los visitantes y ajustar la oferta de productos para optimizar las ventas.

### 3.3 Productos con stock menor a 5 unidades, con categoría y precio

In [ ]:
query3 = """
SELECT
    nombre,
    categoria,
    precio,
    stock
FROM producto
WHERE stock < 5;
"""
df_stock_bajo = pd.read_sql_query(query3, conn)
df_stock_bajo

,nombre,categoria,precio,stock
0,Camiseta de puma,Ropa,13.50,4
1,Camiseta de pantera,Ropa,13.50,2
2,Camiseta misteriosa,Ropa,17.25,4
3,Gorra misteriosa,Ropa,14.99,2
4,Gorra pino,Ropa,15.99,3
5,Roca que parece una cara que parece una roca,Souvenir,30.00,3
6,Diario 3 (réplica),Libro,48.99,1


El análisis de los productos con un stock menor a cinco unidades revela una preocupación importante por la disponibilidad limitada de varios artículos populares en la cabaña del misterio. Predominantemente, los productos de la categoría "Ropa", como las camisetas y gorras temáticas, presentan cantidades bajas, lo que indica una alta demanda o una reposición insuficiente. Asimismo, productos destacados como la "Roca que parece una cara que parece una roca" y el "Diario 3 (réplica)" del género libro y souvenir respectivamente, cuentan con inventarios críticos, lo que podría afectar negativamente la satisfacción del cliente si no se gestionan oportunamente. Este comportamiento sugiere la necesidad de un control más riguroso del inventario y una estrategia de reabastecimiento para evitar la pérdida de ventas por falta de stock, especialmente en productos clave que fortalecen la experiencia temática del lugar.

### 3.4 Reporte de eventos paranormales, su descripción, fecha y visitantes que dejaron testimonio

In [ ]:
query4 = """
SELECT
    e.nombre AS evento,
    e.descripcion,
    e.fecha_evento,
    v.nombre AS visitante,
    b.testimonio
FROM visitaParanormal e
LEFT JOIN bitacoraInvestigacion b ON e.id_evento = b.id_evento
LEFT JOIN visitante v ON b.id_visitante = v.id_visitante
ORDER BY e.fecha_evento;
"""
df_eventos_testimonios = pd.read_sql_query(query4, conn)
df_eventos_testimonios

,evento,descripcion,fecha_evento,visitante,testimonio
0,Luces Flotantes,Luces extrañas flotando cerca del mostrador,2023-06-19,Gideon Alegría,Vi luces extrañas flotando cerca del mostrador...
1,Figura Encapuchada,Aparece una figura misteriosa en el pasillo,2025-03-11 15:22:39,Dipper Pines,"Vi una figura encapuchada en el pasillo, pero ..."
2,Figura Encapuchada,Aparece una figura misteriosa en el pasillo,2025-03-11 15:22:39,Mabel Pines,La figura apareció solo por unos segundos. No ...
3,Figura Encapuchada,Aparece una figura misteriosa en el pasillo,2025-03-11 15:22:39,Pacífica Noroeste,"Vi una figura encapuchada en el pasillo, pero ..."
4,Figura Encapuchada,Aparece una figura misteriosa en el pasillo,2025-03-11 15:22:39,Candy Chiu,La figura encapuchada estaba parada frente a l...
5,Figura Encapuchada,Aparece una figura misteriosa en el pasillo,2025-03-11 15:22:39,Grenda,Estaba viendo hacia la ventana y vi una figura...
6,Ruidos Extraños,Se escuchan ruidos extraños en el sótano,2025-03-15 18:00:00,Gideon Alegría,Escuché ruidos extraños provenientes del sótan...


El reporte de eventos paranormales en la cabaña del misterio muestra una actividad recurrente y variada que ha sido registrada en diferentes fechas y con descripciones detalladas por parte de los visitantes. Entre los fenómenos más frecuentes destacan las “Luces Flotantes” y la aparición de una “Figura Encapuchada” en el pasillo, este último evento con múltiples testimonios simultáneos de varios visitantes, lo que refuerza la credibilidad de la experiencia colectiva. Además, se reportan ruidos extraños en el sótano, que generan inquietud y aumentan la atmósfera de misterio. La variedad y recurrencia de estos eventos sugieren una actividad paranormal constante que atrae a visitantes interesados en fenómenos sobrenaturales, reforzando el carácter único del lugar y su importancia como espacio de investigación y turismo temático.

### 3.5 Total de ventas por categoría de producto

In [ ]:
query5 = """
SELECT
    p.categoria,
    SUM(p.precio * ve.cantidad) AS total_ventas
FROM venta ve
JOIN producto p ON ve.id_producto = p.id_producto
GROUP BY p.categoria
ORDER BY total_ventas DESC;
"""
df_ventas_categoria = pd.read_sql_query(query5, conn)
df_ventas_categoria


,categoria,total_ventas
0,Souvenir,117.75
1,Juguete,85.99
2,Libro,83.99
3,Ropa,44.25


El análisis de las ventas por categoría revela que los souvenirs lideran en términos de ingresos, con un total acumulado de 117.75 unidades monetarias, lo que indica una alta demanda de estos productos relacionados con la temática de la cabaña del misterio. Los juguetes y libros también presentan cifras significativas, con ventas totales de 85.99 y 83.99 respectivamente, reflejando el interés tanto en artículos recreativos como en material de lectura relacionado al entorno paranormal. La categoría de ropa, aunque con un volumen menor de ventas (44.25), mantiene una presencia relevante, posiblemente vinculada a prendas temáticas y recuerdos portátiles. Estos datos permiten identificar las preferencias de los visitantes y orientar estrategias de inventario y promoción hacia las categorías con mayor rendimiento económico.

### 3.6 Visitantes que participaron en eventos paranormales y también realizaron compras, con total gastado

In [ ]:
query6 = """
SELECT
    v.id_visitante,
    v.nombre,
    SUM(p.precio * ve.cantidad) AS total_gastado
FROM visitante v
JOIN bitacoraInvestigacion b ON v.id_visitante = b.id_visitante
JOIN venta ve ON v.id_visitante = ve.id_visitante
JOIN producto p ON ve.id_producto = p.id_producto
GROUP BY v.id_visitante, v.nombre
ORDER BY total_gastado DESC;
"""
df_participantes = pd.read_sql_query(query6, conn)
df_participantes

,id_visitante,nombre,total_gastado
0,9,Grenda,101.75
1,2,Mabel Pines,35.99
2,8,Candy Chiu,25.00
3,1,Dipper Pines,17.25


El listado de visitantes que han participado en eventos paranormales y, al mismo tiempo, han realizado compras muestra una clara relación entre el interés por las experiencias sobrenaturales y el gasto en productos relacionados. Grenda destaca como la visitante que más ha invertido, con un total de 101.75 unidades monetarias, seguida por Mabel Pines con 35.99, Candy Chiu con 25.00 y Dipper Pines con 17.25. Este comportamiento sugiere que los visitantes que están más inmersos en las actividades paranormales tienden también a adquirir souvenirs y productos temáticos, lo que representa una oportunidad para enfocar campañas de marketing y promociones específicas hacia este perfil de cliente, maximizando tanto la experiencia como los ingresos del sitio.

## 3.7 ¿Qué tienen en común los visitantes que han reportado actividad paranormal?

In [ ]:
query7 = """
SELECT
    v.gustos,
    COUNT(*) AS frecuencia
FROM visitante v
JOIN bitacoraInvestigacion b ON v.id_visitante = b.id_visitante
GROUP BY v.gustos
ORDER BY frecuencia DESC;
"""
df_comunes = pd.read_sql_query(query7, conn)
df_comunes

,gustos,frecuencia
0,"{""souvenirs"": true, ""creepy_tours"": true, ""cur...",4
1,"{""souvenirs"": false, ""creepy_tours"": true, ""cu...",2
2,"{""souvenirs"": false, ""creepy_tours"": false, ""c...",1


In [ ]:
query8 = """
SELECT
    v.origen,
    COUNT(*) AS cantidad
FROM visitante v
JOIN bitacoraInvestigacion b ON v.id_visitante = b.id_visitante
GROUP BY v.origen
ORDER BY cantidad DESC;
"""
df_comunes2 = pd.read_sql_query(query8, conn)
df_comunes2

,origen,cantidad
0,Oregón,5
1,California,2


In [ ]:
query9 = """
SELECT
    AVG(v.edad) AS edad_promedio
FROM visitante v
JOIN bitacoraInvestigacion b ON v.id_visitante = b.id_visitante;
"""
df_comunes3 = pd.read_sql_query(query9, conn)
df_comunes3

,edad_promedio
0,12.571429


El análisis de los visitantes que han reportado actividad paranormal revela patrones claros en sus preferencias y características demográficas. Primero, un grupo significativo de estos visitantes comparte un interés marcado en los “creepy tours” y los souvenirs relacionados, como lo indica la frecuencia de 4 registros con ambos gustos activos. Esto evidencia que la experiencia paranormal no solo despierta curiosidad, sino que también influye en sus comportamientos de consumo, orientándolos hacia productos y actividades temáticas. Además, la mayoría de estos visitantes proviene del estado de Oregón, con un total de 5 individuos, seguido por California con 2 visitantes. Esta concentración geográfica sugiere una posible tendencia regional o una influencia cultural específica en la preferencia por actividades paranormales. Finalmente, el promedio de edad de 12.57 años indica que este interés está presente principalmente en un público joven, lo cual puede ser relevante para la planificación de eventos y marketing dirigido a este segmento. Los visitantes que reportan experiencias paranormales comparten gustos específicos por actividades y productos relacionados con el misterio, provienen predominantemente de Oregón y constituyen un grupo joven, lo que ofrece valiosas pistas para diseñar estrategias que optimicen su experiencia y maximicen el impacto comercial.